% CGRtools Tutorial
% Dr. Ramil Nugmanov; Dr. Timur Madzhidov; Ravil Mukhametgaleev
% Mar 25, 2019

# CGRtools Tutorial

(c) 2019, Dr. Ramil Nugmanov; Dr. Timur Madzhidov; Ravil Mukhametgaleev

Installation instructions of CGRtools package information and tutorial's files see on `https://github.com/cimm-kzn/CGRtools`

NOTE: Tutorial should be performed sequentially from the start. Random cell running will lead to unexpected results. 

In [1]:
import pkg_resources
if pkg_resources.get_distribution('CGRtools').version.split('.')[:2] != ['3', '1']:
    print('WARNING. Tutorial was tested on 3.1 version of CGRtools')
else:
    print('Welcome!')

Welcome!


In [2]:
# load data for tutorial
from pickle import load
from traceback import format_exc

with open('molecules.dat', 'rb') as f:
    molecules = load(f) # list of MoleculeContainer objects
with open('reactions.dat', 'rb') as f:
    reactions = load(f) # list of ReactionContainer objects

m1, m2, m3, m4 = molecules # molecule
m7 = m3.copy()
m11 = m3.copy()
m11.standardize()
m7.standardize()
r1 = reactions[0] # reaction
m5 = r1.reactants[0]
m8 = m7.substructure([4, 5, 6, 7, 8, 9], as_view=False)
m10 =  r1.products[0].copy()

# 1. Data types and operations with them

CGRtools has subpackage *containers* with data structures classes:

* *MoleculeContainer* - for molecular structure
* *ReactionContainer* - for chemical reaction 
* *CGRContainer* - for Condensed Graph of Reaction
* *QueryContainer* - queries for substructure search in molecules
* *QueryCGRContainer* - queries for substructure search in CGRs

In [3]:
from CGRtools.containers import * # import all containers

## 1.1. MoleculeContainer
Molecules are represented as undirected graphs. Molecules contain *Atom* objects and *Bond* objects.

*Atom* objects are represented as dictionary with unique number for each atom as key.  

*Bond* objects are stored as sparse matrix with adjacent atoms pair as keys for rows and columns.

Hereafter, *atom number* is unique integer used to enumerate atoms in molecule. Please, don't confuse it with element number in Periodic Table, hereafter called *element number*.

Methods for molecule handling and the arguments of *MoleculeContainer* are described below.

In [4]:
m1.meta # dictionary for molecule properties storage. For example, DTYPE/DATUM fields of SDF file are read into this dictionary

{'cdid': '001'}

In [5]:
m1 # MoleculeContainer supports depiction and graphic representation in Jupyter notebooks.

In [6]:
m1.depict() # depiction returns SVG image in format string

'<svg width="2.43cm" height="1.41cm" viewBox="-11.31 -0.64 2.43 1.41" xmlns="http://www.w3.org/2000/svg" version="1.1">\n  <g font-family="sans-serif">\n    <g fill="#FF0D0D">\n      <text x="-9.52" y="0.00" font-size="0.40">O</text>\n      <text x="-9.78" y="-0.28" font-size="0.30">15</text>\n    </g>\n  </g>\n  <g fill="none" stroke="black" stroke-width=".03">\n    <line x1="-10.83" y1="-0.10" x2="-10.12" y2="0.31" />\n    <line x1="-10.79" y1="-0.17" x2="-10.08" y2="0.24" />\n    <line x1="-10.10" y1="0.28" x2="-9.57" y2="-0.03" />\n  </g>\n</svg>'

In [7]:
with open('molecule.svg', 'w') as f: # saving image to SVG file
    f.write(m1.depict())

In [8]:
m_copy = m1.copy() # isolated copy of molecule
m_copy

In [9]:
len(m1) # get number of atoms in molecule
# or 
m1.atoms_count

3

In [10]:
m1.bonds_count # number of bonds

2

In [11]:
m1.atoms_numbers # list of atoms numbers

[3, 4, 1]

In [12]:
# this method calculates additional atoms attributes: number of neighbors and hybridization. See below for usage
m1.reset_query_marks() # by default this attributes are None (for speed-up)
m3.reset_query_marks()

The following notations are used for hybridization of atoms. Values are given as numbers below (in parenthesis symbols that are used in SMILES-like signatures are shown):

* 1 (s) - all bonds of atom are single, i.e. sp3 hybridization
* 2 (d) - atom has one double bond and others are single, i.e. sp2 hybridization
* 3 (t) - atom has one triple or two double bonds and other are single, i.e. sp hybridization
* 4 (a) - atom is in aromatic ring

Neighbors and hybridizations atom attributes are required for **substructure operations** and structure standardization. See below

In [13]:
# iterate over atoms using its numbers
list(m1.atoms())  # works the same as dict.items()

[(3, <CGRtools.attributes.molecule.Atom at 0x7f714a552748>),
 (4, <CGRtools.attributes.molecule.Atom at 0x7f714a5527c8>),
 (1, <CGRtools.attributes.molecule.Atom at 0x7f714a552848>)]

In [14]:
# iterate over bonds using adjacent atoms numbers
list(m1.bonds())

[(3, 4, <CGRtools.attributes.molecule.Bond at 0x7f7184ae7248>),
 (4, 1, <CGRtools.attributes.molecule.Bond at 0x7f7184ae7288>)]

In [15]:
# access to atom by number
m1.atom(1)

In [16]:
try:
    m1.atom(10) # raise error for absent atom numbers
except KeyError:
    print(format_exc())

Traceback (most recent call last):
  File "<ipython-input-16-88aab1feb874>", line 2, in <module>
    m1.atom(10) # raise error for absent atom numbers
  File "/home/ravil/for_cgr3/lib/python3.7/site-packages/CGRtools/containers/common.py", line 50, in atom
    return self._node[n]
KeyError: 10



In [17]:
# access to bond using adjacent atoms numbers
m1.bond(1, 4)

In [18]:
try:
    m1.bond(1, 3) # raise error for absent bond
except KeyError:
    print(format_exc())

Traceback (most recent call last):
  File "<ipython-input-18-f820f0a0f50e>", line 2, in <module>
    m1.bond(1, 3) # raise error for absent bond
  File "/home/ravil/for_cgr3/lib/python3.7/site-packages/CGRtools/containers/common.py", line 53, in bond
    return self._adj[n][m]
KeyError: 3



#### Atom objects are dictinary-like classes which store information about:
* element
* isotope
* charge
* multiplicity
* xyz coordinates

Also atoms has methods for data integrity checks and include some internally used data.

In [19]:
a = m1.atom(1)

# access to information
a.element # element symbol
# or
a['element']

'O'

In [20]:
a.charge # formal charge
# or
a['charge']

0

In [21]:
a.multiplicity # atom multiplicity. None if not set
# or
a['multiplicity']

In [22]:
a.isotope # atom isotope. Default isotope if not set. Default isotopes are the same as used in InChI notation
# or
a['isotope']

15

In [23]:
a.x # coordinates
a.y
a.z
# or
a['x']
a['y']
a['z']

0.0

In [24]:
a.neighbors # Number of neighboring atoms is calculated by reset_query_marks method as shown above. It is read-only.

1

In [25]:
a.hybridization # Atoms hybridization is calculated by reset_query_marks method as shown above. It is read-only.

1

In [26]:
try:
    a.hybridization = 2 # Not subsettable. Read-only! Thus error is raised.
except AttributeError:
    print(format_exc())

Traceback (most recent call last):
  File "<ipython-input-26-baf385e573c1>", line 2, in <module>
    a.hybridization = 2 # Not subsettable. Read-only! Thus error is raised.
  File "/home/ravil/for_cgr3/lib/python3.7/site-packages/CGRtools/attributes/molecule.py", line 142, in __setattr__
    super().__setattr__(key, value)
AttributeError: can't set attribute



#### Atomic attributes are subsettable.

CGRtools has integrity checks for verification of changes induced by user

In [27]:
a.isotope = 16
# or 
a['isotope'] = 16

In [28]:
m1.flush_cache() # due to caching used for speed-up one needs to reset cache of molecule to observe changes in atoms and bonds 
m1

In [29]:
try:
    a.isotope = 0 # raise error. Isotope with mass 0 could not exist
except ValueError:
    print(format_exc())

Traceback (most recent call last):
  File "<ipython-input-29-4063e3b69c0d>", line 2, in <module>
    a.isotope = 0 # raise error. Isotope with mass 0 could not exist
  File "/home/ravil/for_cgr3/lib/python3.7/site-packages/CGRtools/attributes/molecule.py", line 129, in __setattr__
    value = getattr(self, f'_{key}_check')(value)
  File "/home/ravil/for_cgr3/lib/python3.7/site-packages/CGRtools/attributes/molecule.py", line 75, in _isotope_check
    raise ValueError('invalid isotope')
ValueError: invalid isotope



In [30]:
# bond objects also are dictionary-like classes which store information about bond order
b = m1.bond(3, 4)

b.order
#
b['order']

2

In [31]:
b.order = 1 # order change also possible
# or
b['order'] = 1

In [32]:
m1.flush_cache() #after flushing cashe one could see molecule with changes
m1

In [33]:
try:
    b['order'] = 0 # error raises. Bond order 0 is invalid. To delete bond use method described below
except ValueError:
    print(format_exc())

Traceback (most recent call last):
  File "<ipython-input-33-728729e99a18>", line 2, in <module>
    b['order'] = 0 # error raises. Bond order 0 is invalid. To delete bond use method described below
  File "/home/ravil/for_cgr3/lib/python3.7/site-packages/CGRtools/attributes/molecule.py", line 28, in __setitem__
    setattr(self, key, value)
  File "/home/ravil/for_cgr3/lib/python3.7/site-packages/CGRtools/attributes/molecule.py", line 343, in __setattr__
    value = getattr(self, f'_{key}_check')(value)
  File "/home/ravil/for_cgr3/lib/python3.7/site-packages/CGRtools/attributes/molecule.py", line 389, in _order_check
    raise ValueError('invalid order')
ValueError: invalid order



One should to use `delete_bond` method to break bond

In [34]:
m1.delete_bond(3, 4)
m1                    #Note that cashe flushing is not required here.

Method `delete_atom` removes atom from the molecule

In [35]:
m1.delete_atom(3)
m1

In [36]:
m_copy # copy unchanged!

*Atoms* and *bonds* objects can be converted into integer representation that could be used to classify their types.

*Atom* type is represented by 21 bit code rounded to 32 bit integer number:

* 7 bits stand for atom number (2 ** 7 - 1 == 127, currently 118 elements are presented in Periodic Table)
* 9 bits are used for isotope (511 posibilities, highest known isotope is ~300)
* 3 bits stand for formal charge. Charges range from -3 to +3 rescaled to range 0-6
* 2 bits are used for multiplicity.

In [37]:
int(a)
# 131596 == 0001000 000010000 011 00
# 0001000 == 8 Oxygen
# 000010000 == 16 isotope
# 011 == 3 (3 - 3 = 0) uncharged
# 00 == 0 hasn't multiplicity

131596

In [38]:
int(b)  # bonds are encoded by their order

1

In [39]:
print(m1.atom_implicit_h(1)) # get number of implicit hydrogens on atom 1
print(m1.atom_explicit_h(1)) # get number of explicit hydrogens on atom 1
print(m1.atom_total_h(1)) # get total number of hydrogens on atom 1

1
0
1


In [40]:
m1

In [41]:
m1.check_valence() # return list of numbers of atoms with invalid valences

[]

In [42]:
m4 # molecule with valence errors

In [43]:
m4.check_valence()

[1]

In [44]:
m3

In [45]:
m3.sssr # Method for application of Smallest Set of Smallest Rings algorithm for rings 
        # identification. Returns list of lists of atoms forming smallest rings

[[4, 9, 8, 7, 6, 5]]

#### Connected components.
Sometimes molecules has disconnected components (salts etc).

One can find them and split molecule to separate components.

In [46]:
m2 # it's a salt represented as one graph

In [47]:
m2.connected_components # list of lists of atoms belonging to graph components

[[5, 6, 7, 8, 9, 10], [11]]

In [48]:
anion, cation = m2.split() # split molecule to components

In [49]:
anion # graph of only one salt component

In [50]:
cation # graph of only one salt component

#### Union of molecules
Sometimes it is more convenient to represent salts as ion pair. Otherwise ambiguity could be introduced, for example in reaction of salt exchange:

**Ag+** + **NO3-** + **Na+** + **Br-** = **Ag+** + **Br-** + **Na+** + **NO3-**. Reactants and products sets are the same. 

In this case one can combine anion-cation pair into single graph. It could be convenient way to represent other molecule mixtures.

In [51]:
salt = anion | cation 

# or 

anion.union(cation)

salt # this graph has disconnected components, it is considered single compound now

#### Substructures could be extracted from molecules.

By default, returned substructures are read-only projections of original molecule (except attributes of atoms/bonds). 

Changes in original molecule (bond breaking/formation, atom insertion/deletion, atom/bond attributes changes) will be mirrored in projection.  

Projections share the same neighbors and hybridization attributes as in initial molecule even if could be wrong for substructure 

In [52]:
proj = m3.substructure([4,5,6,7,8,9])  # projection of substructure
proj

In [53]:
m3.atom(4).neighbors

3

In [54]:
proj.atom(4).neighbors # same as in original molecule and not as it should be in substructure

3

In [55]:
from networkx.exception import NetworkXError
try:
    proj.reset_query_marks() # change of structure for projections is blocked
except NetworkXError:
    print(format_exc())

Traceback (most recent call last):
  File "<ipython-input-55-cc1c1d2b39f3>", line 3, in <module>
    proj.reset_query_marks() # change of structure for projections is blocked
  File "/home/ravil/for_cgr3/lib/python3.7/site-packages/networkx/classes/function.py", line 142, in frozen
    raise nx.NetworkXError("Frozen graph can't be modified")
networkx.exception.NetworkXError: Frozen graph can't be modified



In [56]:
benzene = m3.substructure([4,5,6,7,8,9], as_view=False) # Substructure could be extracted as isolated graph (not projection)
benzene

In [57]:
benzene.atom(4).neighbors is None # empty attribute. Substructure is a new molecule here. We need to call reset_query_marks

True

In [58]:
benzene.reset_query_marks()

In [59]:
benzene.atom(4).neighbors # now number of neighbors for atom 4 is 2. It is not 3 as above where projection was used.

2

Note:

* Projection of projection also projection of original molecule
* Projection can be converted to isolated molecule by calling method copy()

In [60]:
proj_copy = proj.copy() # turning projection into molecule using "copy" method
proj_copy

In [61]:
proj_copy.reset_query_marks() # This not a projection any more but a new molecule

Changes in projection are mirrored. See example:

In [62]:
m3.delete_bond(4, 5) # we've deleted bond
m3

In [63]:
proj.flush_cache() # remove cached image. Note that in projection the bond is also deleted.
proj

`augmented_substructure` is a substructure consisting from atoms and a given number of shells of neighboring atoms around it.
**deep** argument is a number of considered shells. 

It also returns projection by default.

In [64]:
aug = m3.augmented_substructure([10], deep=2) #  atom 10 is Nitrogen
aug

In [65]:
aug.atom(10).hybridization #atom has two incident double bond.

3

#### Atoms Ordering.
This functionality is used for canonic numbering of atoms in molecules. Prime number multiplication based Morgan algorithm is used for atom ranking. Property `atoms_order` returns dictionary of atom numbers as keys and their ranks according to canonicalization as values. Equal rank mean that atoms are symmetric (are mapped to each other in automorhisms). In present version, instead of sequential ranks prime numbers are returned.


In [66]:
m5.atoms_order

{3: 2, 1: 3, 4: 5}

#### Atom number can be changed by `remap` method.

This method is useful when it is needed to change order of atoms in molecules. First argument to `remap` method is dictionary with existing atom numbers as keys and desired atom number as values. It is possible to change atom numbers for only part of atoms. Atom numbers could be non-sequencial but need to be unique. 

If argument *copy* is set ***True*** new object will be created, else existing molecule will be changed. Default is ***False***. 

In [67]:
for n, a in m5.atoms():
    print(n, a.element)
for n, m, b in m5.bonds():
    print(m, n, b.order)

3 C
4 C
1 O
4 3 1
1 4 1


In [68]:
remapped = m5.remap({4:2}, copy=True)
remapped

In [69]:
for n, a in remapped.atoms():
    print(n, a.element)
for n, m, b in remapped.bonds():
    print(m, n, b.order)

3 C
2 C
1 O
2 3 1
1 2 1


## 1.2. ReactionContainer

*ReactionContainer* objects has the following properties:

* **reactants** - list of reactants molecules
* **reagents** - list of reagents molecules
* **products** - list of products molecules
* **meta** - dictinary of reaction metadata (DTYPE/DATUM block in RDF)

In [70]:
r1 # depiction supported

In [71]:
r1.meta

{'CdId': '1872',
 'solvent': '3',
 'temperature': '129.5',
 'tabulated_constant': '-6.87'}

In [72]:
print(r1.reactants, r1.products)  # Access to lists of reactant and products. Molecules' signatures are returned by print() method.
reactant1, reactant2, reactant3 = r1.reactants
product = r1.products[0]

[C-C-O, O-C(=O)-C(-O)=O, C-C-[O-]] [C-C-O-C(=O)-C(=O)-O-C-C]


Reactions also has `standardize`, `aromatize`, `reset_query_marks`, `implicify_hydrogens` and `explicify_hydrogens` methods (see part 3). These methods are applied independently to every molecule in reaction.

## 1.3. CGR
*CGRContainer* object is similar to *MoleculeConrtainer*, except some methods. The following methods are not suppoted for *CGRContainer*:

* aromatize
* standardize
* implicify_hydrogens
* explicify_hydrogens
* atom_implicit_h
* atom_explicit_h
* atom_total_h
* check_valence

*CGRContainer* also has some methods absent in *MoleculeConrtainer*:

* centers_list
* center_atoms
* center_bonds

*CGRContainer* is undirected graph. Atoms and bonds in CGR has two states: reactant and product.

#### Composing to CGR

As mentioned above, atoms in *MoleculeContainer* have unique numbers. These numbers are used as atom-to-atom mapping in CGRtools upon CGR creation. Thus, atom order for molecules in reaction should correspond to atom-to-atom mapping.  

Pair of molecules can be transformed into CGR. Notice that, the same atom numbers in reagents and products imply the same atoms.

Reaction also can be composed into CGR. Atom numbers of molecules in reaction are used as atom-to-atom mapping of reactants to products.

In [73]:
cgr1 = m7 ^ m8 # CGR from molecules
# or 
m7.compose(m8)

print(cgr1)  # CGRcontainer object currently doesn't support depiction. CGR signature is printed out.

C:1:C:C(:C:C:C:1-[N+](-[O-])=O)-O>>C:1:C:C(:C:C:C:1.[N+](-[O-])=O).O


This is CGR (depiction is made by ChemAxon externally). You can see changed bonds connected to ring.
![cgr1.png](cgr1.png)

In [74]:
r1

In [75]:
cgr2 = ~r1 # CGR from reactions

# or 

r1.compose()
print(cgr2) # signature is printed out.

C-C-[O-].C(=O)(-O)-C(=O)(-O).O-C-C>>C-C-O-C(=O)(.O)-C(=O)(.O)-O-C-C


It is CGR for reaction (depiction is made by ChemAxon externally).
![cgr2.png](cgr2.png)

In [76]:
cgr2.reset_query_marks() # CGRs also has reset_query_marks method

In [77]:
a = cgr2.atom(2) # atom access is the same as for MoleculeContainer

In [78]:
a.element # element attribute

# or 

a['element']

'O'

In [79]:
a.isotope # isotope attribute

# or 

a['isotope']

16

For *CGRContainer* attributes `charge`, `multiplicity`, `x`, `y`, `z`, `neighbors` and `hybridization` refer to atom state in reactant of reaction;  arguments `p_charge`, `p_multiplicity`, `p_x`, `p_y`, `p_z`, `p_neighbors` and `p_hybridization` could be used to extract atom state in product part in reaction.

In [80]:
a.charge # charge of atom in reactant

# or 

a['charge']

-1

In [81]:
a.p_charge # charge of atom in product

#or 

a['p_charge']

0

In [82]:
a.p_multiplicity # multiplicity of atom in product. It is None and thus not returned

In [83]:
a.p_x  # coordinates of atom in product
a.p_y
a.p_z

0.0

In [84]:
a.neighbors # number of neighbors of atom in reactant

1

In [85]:
a.p_neighbors # number of neighbors of atom in product

2

In [86]:
a.hybridization # hybridization of atom in reactant. 1 means only single bonds are incident to atom

1

In [87]:
a.p_hybridization # hybridization of atom in product. 1 means only single bonds are incident to atom

1

In [88]:
b = cgr1.bond(4, 10) # take bond

#### Bonds has `order` and `p_order` attribute
If `order` attribute value is ***None***, it means that bond was formed  
If `p_order` is ***None***, it means that bond was broken  

Both `order` and `p_order` can't be ***None***

In [89]:
b.order # bond order in reactant

1

In [90]:
b.p_order is None # bond order in product in None

True

**CGR can be decomposed back to reaction**, i.e. reactants and products.

Notice that CGR can lose information in case of unbalanced reactions (where some atoms of reactant does not have counterpart in product, and vice versa). Decomposition of CGRs for unbalanced reactions back to reaction may lead to strange (and erroneous) structures.

In [91]:
reactant_part, product_part = ~cgr1 # CGR of unbalanced reaction is decomposed back into reaction
# or 
cgr1.decompose()

(<CGRtools.containers.molecule.MoleculeContainer at 0x7f7184ab0098>,
 <CGRtools.containers.molecule.MoleculeContainer at 0x7f7184ab0138>)

In [92]:
reactant_part # reactants extracted. One can notice it is initial molecule

In [93]:
product_part #extracted products. Originally benzene was the product.

For decomposition of *CGRContainer* back into *ReactionContainer* `CGRpreparer` class can be used. `CGRpreparer` is callable object

In [94]:
from CGRtools import CGRpreparer # import of CGRpreparer
preparer = CGRpreparer()         # initialization of CGRpreparer

In [95]:
decomposed = preparer.decompose(cgr2) # decomposition of CGR2 into reaction

In [96]:
decomposed # You can see that water absent in products initially was restored. 
# This is a side-effect of CGR decomposing that could help with reaction balancing. 
# But balancing using CGR decomposition works correctly only if minor part atoms are lost 
# but multiplicity and formal charge are saved. In next release electronic state balansing will be added.

In [97]:
r1 # compare with initial reaction

## 1.4 Queries

CGRtools supports special objects for Queries. Queries are designed for substructure isomorphism. User can set number of neighbors and hybridization by himself (in molecules they could be calculated but could not be changed).

Queries don't have `reset_query_marks` method

In [98]:
from CGRtools.containers import*

In [99]:
m10.reset_query_marks()
m10 # ether

In [100]:
carb = m10.substructure([5,7,8, 2]) # extract projection of carboxyl fragment
carb

In [101]:
q = QueryContainer(carb)  # convert fragment into query
print(q) # QueryContainer don't support depiction yet but signatures can be extracted

[O;s2;]-[C;d3;]=[O;d1;]


CGRs also can be transformed into Query.

`QueryCGRContainer` is similar to QueryContainer class for CGRs and has the same API.

`QueryCGRContainer` take into account state of atoms and bonds in reactant and product, including neighbors and hybridization

In [102]:
cgr1.reset_query_marks()        # cgr1 is CGRContaier its labels could be calculated 
cgr_q = QueryCGRContainer(cgr1) # transfrom CGRContainer into QueryCGRContainer
print(cgr_q)                    # print out signature of query

[C;a3;]:1(:[C;a2;]:[C;a2;]:[C;a3;](:[C;a2;]:[C;a2;]:1)-[O;s1;])-[N;d3;+](-[O;s1;-])=[O;d1;]>>[C;a2;]:1(:[C;a2;]:[C;a2;]:[C;a2;](:[C;a2;]:[C;a2;]:1).[O;s0;]).[N;d2;+](-[O;s1;-])=[O;d1;]


## 1.5. Molecules, CGRs, Reactions construction

CGRtools has API for objects construction from scratch.

CGR and Molecule has methods `add_atom` and `add_bond` for adding atoms and bonds.

In [103]:
from CGRtools.containers import *

In [104]:
m = MoleculeContainer() # new empty molecule

m.add_atom('C')  # add Carbon atom using element symbol
m.add_atom(6)    # add Carbon atom using element number. {'element': 6} is not valid, but {'element': 'O'} is also acceptable
m.add_atom({'element': 'O', 'charge': -1}) # add negatively charged Oxygen atom. Similarly other atomic properties can be set

# add_atom has second argument for setting atom number. 
# If not set, the next integer after the biggest among already created will be used.
m.add_atom({'element': 'Na', 'charge': 1}, 4)

4

In [105]:
m.add_bond(1, 2, 1) # add bond with order = 1 between atoms 1 and 2
m.add_bond(3, 2, {'order': 1}) # the other possibility to set bond order

In [106]:
m.calculate2d() #experimental function to calculate atom coordinates. Has number of flaws yet
m

Reactions can be constructed from molecules

In [107]:
r = ReactionContainer() # empty reaction
r.reactants.append(m1) # add reactant
r.products.append(m11)  # add product
# or
r = ReactionContainer(reactants=[m1], products=[m11]) # one-step way to construct reaction
# or
r = ReactionContainer([m1], [m11]) # first list of MoleculeContainers is interpreted as reactants, second one - as products

In [108]:
r

In [109]:
# reactants, products, reagents attributes are list-like. 
r.products.append(m.copy()) # One can add (or remove) molecules directly to this list of products returned by r.products
r.flush_cache()

In [110]:
r # coordinates will left unchanged. Thus depiction could look wrong.

In [111]:
r.fix_positions() # this method fixes coordinates of molecules in reaction
r

*QueryContainers* can be constructed in the same way as *MoleculeContainers*.

Unlike other containers *QueryContainers* additionally support atoms, neighbors and hybridization lists.

In [112]:
q = QueryContainer() # creation of empty container
q.add_atom({'element': ['N', 'O']}) # add atom list: N or O atom, any isotope and multiplicity, neutral, 
                                    # number of neighbors and hybridization are irrelevant
q.add_atom({'element': 'C', 'neighbors': [2, 3]}) # add carbon atom, any isotope and multiplicity, neutral, 
                                                  # has 2 or 3 non-hydrogen heighbors
q.add_atom({'element': 'O'}) # add oxygen atom, any isotope and multiplicity, neutral, 
                             # number of neighbors and hybridization are irrelevant 
q.add_bond(1, 2, 1) # add single bond between atom 1 and 2 
q.add_bond(2, 3, 2) # add double bond between atom 1 and 2 
# any amide or carboxilic group will fit this query

In [113]:
print(q) # print out signature (SMILES-like)

[N,O]-[C;<23>;]=[O]


## 1.6. Extending CGRtools

You can easily customize CGRtools for your tasks.  
CGRtools is OOP-oriented library with subclassing and inheritance support.

As an example, we show how special marks on atoms for ligand donor centers can be added.

In [114]:
from CGRtools.containers import MoleculeContainer
from CGRtools.attributes import Atom

In [115]:
class MarkedAtom(Atom):  # this class will inherite Atom class
    __slots__ = '__mark' # all new attributes should be slotted!
    
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.__mark = None # set default value for added attribute

    @property
    def mark(self):        # created new property 
        return self.__mark
    
    @mark.setter
    def mark(self, mark):
        # do some checks and calculations
        self.__mark = mark

In [116]:
class MarkedMoleculeContainer(MoleculeContainer): # MarkedAtomContainer inherits MoleculeContainer
    node_attr_dict_factory = MarkedAtom # override atom container

In [117]:
m = MarkedMoleculeContainer() # create newly developed container MarkedMoleculeContainer
m.add_atom('C')               # add atom C using methods inherited from MoleculeContainer
m.add_atom('N')               # add atom N
m.add_bond(1, 2, 1)           # add bond

m.atom(2)['mark'] = 1  # set mark on atom. In this example dictinary setitem supported but update not.

In [118]:
m.atom(2).mark # one can return mark

1